In [1]:
import numpy as np
from ipynb.fs.full.Critical_Section import *
import re


In [2]:
'''
pass1 of the algorithm where the events are parsed, CS info is generated, VC for events is parsed.
'''
def pass1(filename):
    events, critical_section_list,n,lock_map = gen_event_cs2(filename)
    # print(lock_map)
    # vc_for_events = generate_vc_for_events(events,n)
    return events,critical_section_list,None,n,lock_map    

'''
Checks if the two entries are CP ordered.
'''
def is_cp(cs_index1,cs_index2,cs):
    if(len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].read_objects)))>0 or 
    len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].write_objects)))>0 or 
    len(list(cs[cs_index1].read_objects.intersection(cs[cs_index2].write_objects)))>0):
        return True
    
    
# TODO Mark CP and HB matrices at one point only  
'''
Marks 1 for each value of  matrix[arr1][arr2] 
'''
def mark_matrix(matrix,arr1,arr2):
    for val in arr1:
        for val2 in arr2:
            matrix[val][val2] = 1
    return matrix
    
'''
Generate CP matrix using rule A in CP paper that the variables must be conflicting for each lock section. 
'''    
def generate_hb_cp_matrix_using_rulea(events,critical_section_list,lock_map):   
    hb_matrix = np.zeros((len(events),len(events)))
    cp_matrix = np.zeros((len(events),len(events)))
    for key, value in lock_map.items():
        for iter_var in range(len(value)):
            for inner_iter_var in range(iter_var+1,len(value)):
                hb_matrix = mark_matrix(hb_matrix,critical_section_list[value[iter_var]].event_idx,critical_section_list[value[inner_iter_var]].event_idx)
                if is_cp(iter_var,inner_iter_var,critical_section_list):
                    cp_matrix = mark_matrix(cp_matrix,critical_section_list[value[iter_var]].event_idx,critical_section_list[value[inner_iter_var]].event_idx)
    return cp_matrix,hb_matrix


'''
Pass2 which generates the cp matrix using rulea.
Also fills the HB matrix 
'''                
def pass2(events,critical_section_list,lock_map):
    cp_matrix,hb_matrix = generate_hb_cp_matrix_using_rulea(events,critical_section_list,lock_map)
    return cp_matrix,hb_matrix


def compute_rule2_rule3(hb_matrix,cp_matrix):
    # start from n - 1 
    for i in reversed(range(hb_matrix.shape[0]-1)):
        for j in (range(hb_matrix.shape[0],i,-1)):
    #         for k in range(j+1,end+1):
              end = hb_matrix.shape[0]
              for k in range(end-1,j,-1):
                if hb_matrix[i][j] == 1 and cp_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1
                elif cp_matrix[i][j] == 1 and hb_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1  
    return hb_matrix,cp_matrix


def are_events_conflicting(e1,e2):
    if(e1.tid!=e2.tid and
        ((e1.e_type==0 and e2.e_type==1) or
       (e1.e_type==1 and e2.e_type==0) or
       (e1.e_type==1 and e2.e_type==1)) and 
       e1.var_name==e2.var_name):

        return True
    else:
        return False
    
       
def detect_races(cp_matrix,events):
    is_race_detected  = False
    for i in range(cp_matrix.shape[0]):
        for j in range(i+1,cp_matrix.shape[0]):
            if cp_matrix[i][j]==0 and are_events_conflicting(events[i],events[j]):
                print('Race on events:',i,j,' Variable name: ',events[i].var_name)
                is_race_detected = True
    if not is_race_detected:
        print('No Race')
        
        
def CSv1(filename):
    print('-'*100)
    print('Reading log file ' + filename)
    events, critical_section_list,vc_for_events,no_of_threads,lock_map = pass1(filename)
    
    cp_matrix,hb_matrix = pass2(events,critical_section_list,lock_map)
    cp_matrix,hb_matrix = compute_rule2_rule3(hb_matrix,cp_matrix)
    detect_races(cp_matrix,events)
    
def iter_cs(critical_section_list):
    for cs in critical_section_list:
        print(cs)